# Spartan Quest

## Purpose
To illustrate the advantages and limitations of search algorithms.
## Author:   Rula Khayrallah
Copyright ©  Rula Khayrallah
## Warning
**Do not make any modifications to this notebook.**
## Content
In this Jupyter notebook, we have all the components needed to represent and visulalize the quest.


## Installs and Imports

In [ ]:
!pip install import-ipynb

In [ ]:
!pip install ipycanvas

In [ ]:
import import_ipynb
import time
import os
from ipycanvas import hold_canvas, MultiCanvas
from ipywidgets import Image

## Main Function: _gospartans_

In [ ]:
def gospartans(maze_name, search,  heuristic=None):        
    if not valid_arguments(maze_name, search, heuristic):
        return
    quest = Problem(maze_name)  # Initialize our search problem for this quest
    search_function = globals()[search]
    start_time = time.time()
    if search == "astar":  # for informed search
         heuristic_function = globals()[heuristic]
         solution = search_function(quest, heuristic_function)
    else:
        solution = search_function(quest)  # Invoke the uninformed search algorithm specified
    elapsed_time = time.time() - start_time

    # Print some statistics
    if solution is not None:
        print('Path length: ', len(solution))
        print('Carrots consumed: ', sum(quest.cost[action] for action in solution))
    else:
        print('The quest failed!')
    print(f'Number of nodes expanded: {quest.nodes_expanded():,}')
    print(f'Processing time: {elapsed_time:.4f}(sec)')

    Visualization(quest, solution)  # Visualize the solution

def valid_arguments(maze_name, search, heuristic=None):
    """
    Validate the arguments passed to the main function
    :param
     maze_name: text file name that contains the maze representation
     search: one of the search algorithms
     heuristic: name of heristic function - only applicable for informed search
    :return: Boolean
    """
    algorithms_supported = globals()
    heuristics = globals()
    valid = True
    if not os.path.isfile(maze_name):
        valid = False
        print("Invalid maze name:", maze_name)
    if search not in algorithms_supported:
        valid = False
        print("Invalid search algorithm:", search)
    if heuristic is not None and heuristic not in heuristics:
        valid = False
        print("Invalid heuristic:", heuristic)
        
    return valid




## Problem Representation
The Problem class is used to represent the quest as a search problem.  
For the uninformed search implementation, you will need to use the following methods (only):
- _start_state_
- _is_goal_
- _expand_

For the heuristic implementation (informed search), you may want to use the following class variables: NORTH, SOUTH, EAST, WEST and cost.  You can access them through the _problem_ object as problem.NORTH, problem.SOUTH, problem.EAST, problem.WEST and problem.cost. 

In [ ]:
class Problem:
    """
    Represent our search problem at any point in the quest

    Arguments:
    mazename (string): name of text file containing the maze info

    Attributes:
    maze (Maze object):  the maze for this quest
    The maze is constant throughout the quest
    mascot_position (tuple of integers): the current position of Sammy
    medals (a set of tuples): a set containing the positions of the
    remaining medals in the quest
    """
    NORTH = "N"
    SOUTH = "S"
    EAST = "E"
    WEST = "W"
    moves = {EAST: (1, 0), WEST: (-1, 0), SOUTH: (0, 1), NORTH: (0, -1)}

    # The cost (number of carrots consumed) associated with each move.
    cost = {EAST: 15, WEST: 1, SOUTH: 2, NORTH: 14}

    def __init__(self, mazename):
        self._nodes_expanded = 0 # private variable
        self.medals = set()
        self.read_quest(mazename)

    def read_quest(self, mazename):
        """
        Read the maze file specified and save the information in a Maze
        object.
        The length of the first line in the file is used to determine
        the width of the maze.  So the last position in the first row of
        the maze must be represented by a character in the first line
        (not a space).
        W or w:  represent a wall
        M or m: represent the presence of a medal at that position
        S or s: represent the starting position of our mascot Sammy
        Any other character: a vacant maze position
        :param
        mazefile (file object): the file object containing the maze info
        :return: None
        """

        try:
            with open(mazename, 'r', encoding='utf-8') as mazefile:
                layout = mazefile.readlines()
                width = len(layout[0].strip()) # the first line
                height = len(layout) # the number of lines represents the height
                self.maze = Maze(width, height)
                y = 0
                for line in layout:
                    x = 0
                    for char in line:
                        if char in {'W', 'w'}: # W represents a wall
                            self.maze.add_wall((x, y))
                        elif char in {'M', 'm'}: # M represents a medal position
                            self.add_medal((x, y))
                        elif char in {'S', 's'}: # S represents Sammy's position
                            self.add_mascot((x, y))
                        x += 1 # anything else is a vacant maze position
                    y += 1
                
        except FileNotFoundError as error:
            print(error)

    def add_mascot(self, position):
        """
        Save the mascot's position
        :param position: tuple (row,column) representing a maze position
        :return: None
        """
        self.mascot_position = position

    def add_medal(self, position):
        """
        Add the specified position to the set containing all the medals
        :param position: tuple (x, y) representing a maze position
        :return: None
        """
        self.medals.add(position)

    def is_goal(self, state):
        """
        Is the state specified a goal state?
        The state is a goal state when there are no medals left to
        collect.
        :param
        state - A state is represented by a tuple containing two tuples:
                the current position (x, y) of Sammy the Spartan
                a tuple containing the positions of the remaining medals

        :return: Boolean - True if this is a goal state, False otherwise
        """
        position, medals_left = state
        return not medals_left

    def start_state(self):
        """
        Return the start state in this quest
        The start state is identified by the mascot's position and the
        initial distribution of the medals in the maze.
        It is represented by a tuple containing two other tuples:
                the current position (x, y) of Sammy the Spartan
                a tuple containing the positions of the medals

        :return:
        state - The start state in the quest
                A state is represented by a tuple containing two tuples:
                the current position (row, column) of Sammy the Spartan
                a tuple containing the positions of the remaining medals
        """
        return self.mascot_position, tuple(self.medals)

    def expand(self, state):
        """
        Return a list of tuples representing all states reachable
        from the current state with their corresponding action and costs
        :param
        state - A state is represented by a tuple containing two other tuples:
                the current position (x, y) of Sammy the Spartan
                a tuple containing the positions of the remaining medals
        :return:
        a list of tuples representing all states that are reachable
        from the current state with their corresponding action and cost
        """
        result = []
        self._nodes_expanded += 1 # update private variable
        position, current_medals = state
        current_x, current_y = position
        for action in self.moves:
            new_position = (current_x + self.moves[action][0], current_y + self.moves[action][1])
            # if the move in that direction is valid
            if self.maze.within_bounds(new_position) and \
                not self.maze.is_wall(new_position):
                new_medals = set(current_medals) - {new_position}
                new_state = (new_position, tuple(new_medals))
                result.append((new_state, action, self.cost[action]))
        return result

    def nodes_expanded(self):
        return self._nodes_expanded

## Maze Representation
The Maze class is used to represent the maze layout: its width, height and walls wall.   
You don't need to access it directly.  


In [ ]:
class Maze:
    """
    Represent the maze layout: its width, height and walls

    Arguments:
    width (int):  the width of the maze
    height (int): the height of the maze

    Attributes:
    width (int):  the width of the maze
    height (int): the height of the maze
    walls (two dimensional list of booleans):
        Each element represents a position in the maze.
        True indicates that there is a wall in that position.
        False indicates the absence of a wall.
        self.walls[x][y] indicates the presence or absence of a wall
        at position (x, y) in the maze.
    """

    def __init__(self, width, height):
        self.walls = [[False for x in range(width)] for y in range(height)]
        self.width = width
        self.height = height

    def add_wall(self, position):
        """
        Add a wall in the specified position
        :param position: tuple(row, column) representing a maze position
        :return: None
        """
        x, y = position
        self.walls[y][x] = True

    def is_wall(self, position):
        """
        Is there a wall in the given position?
        :param position: tuple(row, column) representing a maze position
        :return: (Boolean) True if there is a wall in that position,
        False otherwise
        """
        x, y = position
        return self.walls[y][x]

    def within_bounds(self, position):
        """
        Is the given position within the maze?
        :param position: tuple (row, column) representing a position
        :return: (Boolean) True if the position is inside the maze,
        False otherwise
        """
        x, y = position
        return (0 <= x <= self.width -1) and (0 <= y <= self.height - 1)

## Visualization
This class is used to visualize the quest and its solution.  You don't need to access it directly. 

In [ ]:
class Visualization:
    """
    Visualization of a given solution to a quest

    Arguments:
    problem (a Problem object) representing the quest
            see Problem class definition 
    solution:  list of actions representing the solution to the quest

    Attributes:
    problem (a Problem object) representing the quest
            see Problem class definition
    canvas:  multicanvas used to visualize the solution
    mascot: image representing Sammy the Spartan

    """
    time_interval = 0.1  # decrease time_interval for faster animation
    size = 40  # self.size in pixel for each grid position


    def __init__(self, problem, solution):  
        self.problem = problem
        self.canvas = MultiCanvas(width=self.size * problem.maze.width,
                                       height=self.size * problem.maze.height)
        self.canvas.layout.width = "100%"
        self.canvas.layout.height = "auto"
        background =  self.canvas[0]
        foreground =  self.canvas[-1]

        sammy_x, sammy_y = problem.mascot_position
        for y in range(problem.maze.height):
            for x in range(problem.maze.width):
                if problem.maze.is_wall((x, y)):
                    background.fill_style = 'blue'
                else:
                    background.fill_style = 'black'
                background.fill_rect(x * self.size, y * self.size,
                                        (x + 1) * self.size, (y + 1) * self.size)
        # load the  image file
        self.mascot = Image.from_file("Sammy.gif")
        foreground.draw_image(self.mascot, (sammy_x ) * self.size,
                                               (sammy_y ) * self.size)
        foreground.fill_style = 'gold'
        for each_medal_x, each_medal_y in problem.medals:
            foreground.fill_circle((each_medal_x + 0.5) * self.size,
                                   (each_medal_y + 0.5) * self.size,
                                   0.24 * self.size)

        display(self.canvas)
        if solution is not None:                
            self.animate(solution)



    def animate(self, solution):
        """"
        Show Sammy's moves corresponding to the solution
        :param
        solution (list) - A list of actions/moves
        :return None
        """
        background = self.canvas[0]
        foreground = self.canvas[-1]
        background.stroke_style = "yellow"
        background.fill_style = "yellow"
        x, y = self.problem.mascot_position
        for action in (solution):
          with hold_canvas():          
            # Clear the old animation step
            foreground.clear()         
            new_x = x + self.problem.moves[action][0]
            new_y = y + self.problem.moves[action][1]
            position = (new_x, new_y)

            if not self.problem.maze.within_bounds(position):
                raise Exception('Falling off the maze....')
            elif self.problem.maze.is_wall(position):
                raise Exception('Crash!  Wall encountered')
            elif position in self.problem.medals:
                self.problem.medals.discard(position)

            background.stroke_line((x + 0.5) * self.size,
                                   (y + 0.5) * self.size,
                                   (new_x + 0.5) * self.size,
                                   (new_y + 0.5) * self.size)
            self.draw_arrow(x, y, new_x, new_y)


            foreground.draw_image(self.mascot, 
                             new_x * self.size,
                             new_y * self.size)
            
            for each_medal_x, each_medal_y in self.problem.medals:
                foreground.fill_circle((each_medal_x + 0.5) * self.size,
                                       (each_medal_y + 0.5) * self.size,
                                       0.24 * self.size)
            x, y = new_x, new_y
            time.sleep(self.time_interval)

    def draw_arrow(self, from_x, from_y, to_x, to_y):
        """
        Draw Arrow on the canvas to show direction of move.
        :param
        from_x: original x coordinate
        from_y: original y coordinate
        to_x: target x coordinate
        from_y: target y coordinate
        :return None
        
        """
        base = 0.4
        tip = 0.6

        # set defaults
        arrow_x1 = to_x + base
        arrow_y1 = to_y + base
        arrow_x2 = to_x + base
        arrow_y2 = to_y + base

        if to_x   > from_x: # pointing east
            arrow_y2 = to_y + tip
        elif to_x < from_x: # pointing west
            arrow_x1 = to_x + tip
            arrow_y1 = to_y + tip
            arrow_x2 = to_x + tip

        elif to_y > from_y:  # pointing south
            arrow_x2 = to_x +tip
        else:
            arrow_x1 = to_x + tip
            arrow_y1 = to_y + tip              
            arrow_y2 =to_y + tip

        self.canvas[0].fill_polygon([((to_x + 0.5) * self.size, (to_y + 0.5) * self.size), 
                                (arrow_x1 * self.size, arrow_y1 * self.size),
                                 (arrow_x2 * self.size, arrow_y2 * self.size)])